# Lead Company-Sector Mapping

 - Data Exploration and Preprocessing Script

In [14]:
# !pip install fasttext
# !pip install fasttext

In [15]:
# importing the required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.offline as py
import plotly.graph_objs as go
import seaborn as sns
import plotly.express as px

import re
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
stop = stopwords.words('english')
import fasttext
import fasttext.util
ft = fasttext.load_model('../../clustering_func_div/cc.en.300.bin')
from sklearn.decomposition import PCA
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

In [16]:
# reading data
# data_train = pd.read_csv("../data/6lakh_pipeline_company_companies_data_2021-07-21.csv")
# data=pd.read_csv("../data/50k_data_crunchbase.csv")
X_train=pd.read_csv('../scripts/X_train_raw.csv', lineterminator='\n')
# X_test=pd.read_csv('../scripts/X_test_raw.csv')
y_train=pd.read_csv('../scripts/y_train_raw.csv', lineterminator='\n')
y_test=pd.read_csv('../scripts/y_test_raw.csv', lineterminator='\n')


X_test = pd.read_csv('../scripts/X_test_raw.csv',
                 lineterminator='\n')

In [17]:
X_train.shape

(37544, 53)

In [18]:
X_test.shape

(12515, 53)

In [19]:
y_train.shape

(37544, 1)

In [20]:
y_test.shape

(12515, 1)

In [21]:
# creating and saving a copy
# original_data = data.copy()

# 1. Variable Identification

In [25]:
def preprocessing(df):
    
    df = df[["company_city", "company_country", "company_state", "company_name", "company_products_services", 
             "company_size", "company_type", "company_founded_at", "company_continent", "company_address", "revenue_range", 
             "rank","_id", "company_funding", "lead_count", "job_count", "cb_id", "linkedin_employee_count", "linkedin_follower_count", 
             "company_description", "company_latest_funding_date", "created_at", "company_sector"]]
    
    return df

In [26]:
data_train = preprocessing(X_train)
data_test = preprocessing(X_test)

In [27]:
data_train.head()

,company_city,company_country,company_state,company_name,company_products_services,company_size,company_type,company_founded_at,company_continent,company_address,...,company_funding,lead_count,job_count,cb_id,linkedin_employee_count,linkedin_follower_count,company_description,company_latest_funding_date,created_at,company_sector
0,Barcelona,Spain,Catalonia,Duel4it,[Social Games],1-10,Sole Proprietorship,2013.0,Europe,Av. Ernest Lluch 32,...,0.0,0,0,f29bcff8-b647-d3b4-0b05-9e867ecc6275,1.0,12.0,"Welcome to Duel4it, a dueling game set in a vi...",0.000000e+00,1547273431,Professional Services
1,Chicago,United States,Illinois,"PUSH Wellness Solutions, Inc","[wellness, incentives, behavior change, corpor...",11-50,Private,2011.0,North America,180 N Lasalle St. Suite 2305,...,1801391.0,12,0,a5472171-ec67-2dbe-80e4-3a44171fb70c,11.0,696.0,PUSH is the first fair and effective progress-...,1.536106e+09,1547273431,Healthcare and Lifesciences
2,Stockholm,Sweden,Stockholms Lan,Enecto Sverige AB - turning visitors into busi...,"[Lead generation, Web visitor demographics, We...",1-10,Public,2002.0,Europe,"Hornsgatan 79, 1 Tr",...,0.0,14,0,920bdf2f-ad29-b260-7d0f-3da3e87558db,14.0,181.0,"By utilizing Enecto Web Lead, Enecto’s clients...",0.000000e+00,1547273431,Technology
3,Cape May Point,United States,New Jersey,Homestead Real Estate,NaN,1-10,Private,NaN,North America,502 Bank Street,...,0.0,60,0,f35f4876-f5c8-addc-7ca7-c62910fdd8e9,39.0,NaN,Homestead Real Estate sells all types of real ...,0.000000e+00,1547273431,Professional Services
4,San Francisco,United States,California,FunnelBeam,"[account based sales, ABM, AI, Big Data, Accou...",11-50,Private,2017.0,North America,City And County Of San Francisco,...,0.0,27,0,abbcbae6-f8e3-1be4-3951-ff1366c5eb8b,16.0,NaN,It’s no secret that B2B enterprise sales is ex...,1.487722e+09,1547273431,Technology


In [28]:
data_test.head()

,company_city,company_country,company_state,company_name,company_products_services,company_size,company_type,company_founded_at,company_continent,company_address,...,company_funding,lead_count,job_count,cb_id,linkedin_employee_count,linkedin_follower_count,company_description,company_latest_funding_date,created_at,company_sector
0,Sterling,United States,Virginia,Illumai,"[shampoo, conditioner, finishing spray, hairca...",1-10,Private,2013.0,North America,"46400 Benedict Dr., Suite 203",...,1300000.0,2,0,35fdbabf-a6da-f66b-5521-1d1852e59811,7.0,48.0,The illumai Biome Care System – is the only ha...,1.449965e+09,1547273431,Retail and CPG
1,San Diego,United States,California,Integrity Plus Property Management,NaN,1-10,NaN,2010.0,North America,P.o. Box 16994,...,NaN,9,0,b487117a-c9b8-4bce-a8be-e469de9a365a,NaN,NaN,Integrity Plus Property Management is a full s...,NaN,1537630537,Professional Services
2,Belmont,Australia,Victoria,Mappt,NaN,1-10,Private,2013.0,Oceania,"Takor Group Ltd Unit 28, 12 Cowcher Place",...,0.0,0,0,76e2cc00-fd13-874e-757e-78c827aefaca,NaN,115.0,Mappt is a powerful mobile GIS and data collec...,0.000000e+00,1547273431,Technology
3,Cambridge,United States,Massachusetts,"Jounce Therapeutics, Inc","[Cancer Immunotherapy, Immunology, Oncology, A...",51-200,Public,2013.0,North America,1030 Massachusetts Avenue,...,194249995.0,157,0,35998e83-7a9a-c4e6-2888-3722c1a486ed,151.0,7875.0,"Jounce Therapeutics, Inc. is a clinical-stage ...",1.615334e+09,1547273431,Healthcare and Lifesciences
4,Chorleywood,United Kingdom,Buckinghamshire,Cache Digital,"[Search Engine Optimisation, Search Engine Mar...",1-10,Private,2013.0,Europe,Globe House 25-27 Lower Road,...,NaN,4,0,b74316db-b6d5-41f3-ab13-114cdc41ac1c,3.0,16.0,Cache Digital are an Internet marketing and SE...,NaN,1537596090,Technology


### 3.3 Text Data

In [ ]:
print(" Text Type is: \n\n",type(data["company_description"][0]),"\n\n Text: \n",data["company_description"][0])

In [ ]:
data['company_industry'].value_counts(normalize=True)*100   # percentage wise

In [ ]:
# sns.pairplot(data, hue='company_industry', height=2)
# data['company_industry'].value_counts(normalize=True)*100   # percentage wise
data['company_industry'].value_counts(normalize=True).plot.bar(title = "company_industry")

can treat as `IMBALANCE DATASET` though not high imbalance is there

- `FILLING MISSING VALUES`

In [ ]:
# checking missing data percentage in data
total = data.isnull().sum().sort_values(ascending = False)
percent = (data.isnull().sum()/data.isnull().count()*100).sort_values(ascending = False)
missing_Data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
print(missing_Data)

In [ ]:
data['linkedin_follower_count'].fillna(data['linkedin_follower_count'].median(), inplace=True)
data['company_latest_funding_date'].fillna(data['company_latest_funding_date'].mode()[0], inplace=True)
data['company_funding'].fillna(data['company_funding'].median(), inplace=True)
data['company_founded_at'].fillna(data['company_founded_at'].mode()[0], inplace=True)
data['company_type'].fillna(data['company_type'].mode()[0], inplace=True)
data['linkedin_employee_count'].fillna(data['linkedin_employee_count'].median(), inplace=True)
data['company_address'].fillna(data['company_address'].mode()[0], inplace=True)
data['company_size'].fillna(data['company_size'].mode()[0], inplace=True)
data['company_state'].fillna(data['company_state'].mode()[0], inplace=True)
data['company_city'].fillna(data['company_city'].mode()[0], inplace=True)
data['company_country'].fillna(data['company_country'].mode()[0], inplace=True)
data['company_continent'].fillna(data['company_continent'].mode()[0], inplace=True)
data['company_description'].fillna(data['company_description'].mode()[0], inplace=True)
data['company_continent'].fillna(data['company_continent'].mode()[0], inplace=True)
data['rank'].fillna(data['rank'].median(), inplace=True)
data['company_name'].fillna(data['company_name'].mode()[0], inplace=True)

In [ ]:
data['company_products_services'].fillna(data['company_products_services'].mode()[0], inplace=True)

In [ ]:
# DEALING WITH IMBALANCE DATASET

In [ ]:
def convert_format(n):
    # strip brackets
    n_split_clean = n.strip('[]')
    # split by comma
    n_split = n_split_clean.split(",")
    #strip whitespaces of start and end
    n_split_list =[]
    for i in n_split:
        n_split_final = i.strip(" ")
        n_split_list.append(n_split_final)
    n_new = n_split_list
        
    return n_new

In [ ]:
# DEALING WITH company_products_services

In [ ]:
# data['company_products_services_embeddings'] = data.apply(lambda x: get_embeddings(x["company_products_services"], ft), axis = 1)
data["company_products_services"] = data.apply(lambda x: convert_format(x["company_products_services"]), axis = 1)

In [ ]:
data["company_products_services"][10]

In [ ]:
# best way is to give them as embedding
# [embedding1,embedding2....] ----> [PCA_combined]

# Now fill theese missing values with PCA_combined as variable and drop company_products_services

In [ ]:
# def get_embeddings(words, ft):
#     pca_list = []
#     for word in words:
        
#         word = str(word)
#         f = ft.get_word_vector(word)
        
#         pca = PCA(n_components=1)
#         f = f.reshape(-1,1)
#         pca_comp = pca.fit_transform(f)
#         pca_list.append(pca_comp)
        
        
#     pca_2 = PCA(n_components=1)
#     pca_list = np.array(pca_list).reshape(-1,1)
#     pca_final = pca_2.fit_transform(pca_list)
#     return pca_final

In [ ]:
# !pip install gensim

In [ ]:
%%time
print("YES")

In [ ]:
%%time
# data['company_products_services_embeddings'] = data.apply(lambda x: get_embeddings(x["company_products_services"], ft), axis = 1)
# feat_cols = ['feature'+str(i) for i in range(len(df_designation["designation_embeddings"][0]))]
# # split array entries into individual columns
# df_designation[feat_cols] = pd.DataFrame(df_designation.designation_embeddings.tolist(), index= df_designation.index)

In [ ]:
# ft.get_dimension()
# fasttext.util.reduce_model(ft, 10)

In [ ]:
# fasttext.util.reduce_model(ft, 200)
data.head()

In [ ]:

# def get_embeddings(word_list, ft):
#     embedding_list=[]
#     for word in word_list:
        
#         word = str(word)
#         f = ft.get_word_vector(word)
#         embedding_list.append(f)

#     return embedding_list

In [ ]:
# data['company_products_services_embeddings'] = data.apply(lambda x: get_embeddings(x["company_products_services"], ft), axis = 1)

### 4. Log Transformations-removing outliers/skewness

In [ ]:
# Make Transformation for required features to remove outlier and skewness

### 5. FEATURE ENGINEERING

In [ ]:
data.head()

In [ ]:
# feature from datetime

### 6. TEXT CLEANING/Pre-Processing

In [ ]:
data['company_description'] = data['company_description'].apply(lambda x: "".join(re.sub(r"[^A-Za-z\s]", '',str(x))))

In [ ]:
data['company_description'] = data['company_description'].apply(lambda x: " ".join(x.lower() for x in x.split()))

In [ ]:
#Removing punctuations
#adding characters list which needs to remove that is PUNCTUATION
punc = ['.', ',', '"', "'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}',"%",'/','<','>','br','�','^','XX','XXXX','xxxx','xx']
#removing extra characters
data['company_description'] = data['company_description'].apply(lambda x: " ".join(x for x in x.split() if x not in punc))

In [ ]:
#removal of stopwords
data['company_description'] = data['company_description'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

In [ ]:
#STEMMING
st = PorterStemmer()
data['company_description']=data['company_description'].apply(lambda x: " ".join([st.stem(w) for w in x.split()]))

In [ ]:
data['company_description'][0], data['company_description'][10], data['company_description'][100]

In [ ]:
# dropping company_products_services for now

### 7. CONVERT INTO EMBEDDINGS

### 8. SCALING

### 9. MODEL FIT